<a href="https://colab.research.google.com/github/idptools/idpcolab/blob/main/arbitrary_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Software Installations

In [11]:
!pip install git+https://github.com/idptools/goose@arbitrary_optimization

  Cloning https://github.com/idptools/goose (to revision arbitrary_optimization) to /tmp/pip-req-build-jjx42i9z
  Running command git clone --filter=blob:none --quiet https://github.com/idptools/goose /tmp/pip-req-build-jjx42i9z
  Running command git checkout -b arbitrary_optimization --track origin/arbitrary_optimization
  Switched to a new branch 'arbitrary_optimization'
  Branch 'arbitrary_optimization' set up to track remote branch 'arbitrary_optimization' from 'origin'.
  Resolved https://github.com/idptools/goose to commit f13a3448486cb6b34c7ab88cab3e16fc4caca301
  Preparing metadata (setup.py) ... done
  Cloning https://****@github.com/idptools/sparrow.git to /tmp/pip-install-mcdiet34/sparrow_f6ebc4b8a41248b68ad30be98940d902
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/idptools/sparrow.git' /tmp/pip-install-mcdiet34/sparrow_f6ebc4b8a41248b68ad30be98940d902
  Resolved https://****@github.com/idptools/sparrow.git to commit b8ba08dddd31f7aaf499126

# File imports

In [1]:
import logging
logging.basicConfig(
    level=logging.INFO, # allow INFO level messages to pass through the logger
    force=True, # Resets any previous configuration
)

from goose.properties import (
    FCR,
    NCPR,
    SCD,
    SHD,
    Complexity,
    ComputeIWD,
    EndToEndDistance,
    Hydrophobicity,
    Kappa,
    ProteinProperty,
    RadiusOfGyration,
    TargetAminoAcidFractions,
)

from goose.sequence_optimization import SequenceOptimizer

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Error importing GPy.
 If trying to run parrot-optimize, make sure to use `pip install idptools-parrot[optimize]`


# An Introduction to the `ProteinProperty` Base Class: defining your own custom parameters for optimization.

The arbitrary sequence optimization interface for GOOSE fundamentally relies upon the following `ProteinProperty` base class. Let's walk through the components of the following base class so that things are explicit and clear.


## Base classes
A base class (also known as a parent class or superclass) is a class in object-oriented programming from which other classes (called derived classes, child classes, or subclasses) can inherit properties and methods.

The primary purpose of a base class is to provide common attributes and behaviors that can be shared by multiple derived classes, allowing for code reuse and the creation of a hierarchical class structure.

**Key Points**
- Inheritance: Derived classes inherit attributes and methods from the base class, which helps avoid code duplication.
- Common Functionality: The base class defines common functionality that can be used or overridden by derived classes.
- Abstract Base Class: A special type of base class that is not meant to be instantiated directly but is designed to be subclassed. It can include abstract methods that must be implemented by derived classes.

Here is our complete `ProteinProperty` base class, but we'll walk through each component of this class.
```
class ProteinProperty(ABC):
    """
    Abstract base class for protein sequence properties to be optimized.
    """
    def __init__(self, target_value: float, weight: float = 1.0):
        assert isinstance(target_value, (int,float)), f"target_value must be numerical. Received {type(target_value)}"
        assert isinstance(weight, (int,float)), f"Weight must be numerical. Received {type(weight)}"
        self._target_value = target_value
        self._weight = weight

    @property
    def target_value(self) -> float:
        return self._target_value

    @target_value.setter
    def target_value(self, value: float):
        assert isinstance(target_value, (int,float)), f"target_value must be numerical. Received {type(value)}"
        self._target_value = value

    @property
    def weight(self) -> float:
        return self._weight

    @weight.setter
    def weight(self, value: float):
        assert isinstance(value, (int,float)), f"weight must be numerical. Received {type(value)}"
        self._weight = value

    @abstractmethod
    def calculate(self, protein: 'sparrow.Protein') -> float:
        """
        Calculate the property value for a given protein.

        Parameters:
        protein (sparrow.Protein): The protein instance.

        Returns:
        float: The calculated property value.
        """
        pass
```
The base class defined here is a special base class known as the abstract base class, this will provide the interface for arbitrarily complex new objectives that adhere to the same underlying interface. This will serve as a useful feature for performing our optimization later


Firstly, we have the class' `init` function. This function defines the `target_value` and a `weight` both of which must be a numerical datatype.



```
class ProteinProperty(ABC):
    """
    Abstract base class for protein sequence properties to be optimized.
    """
    def __init__(self, target_value: float, weight: float = 1.0):
        assert isinstance(target_value, (int,float)), f"target_value must be numerical. Received {type(target_value)}"
        assert isinstance(weight, (int,float)), f"Weight must be numerical. Received {type(weight)}"
        self._target_value = target_value
        self._weight = weight
```




The parameters define both the 'numerical objective' that we will be optimizing towards (more on this later), and the weight - or the emphasis - which we will place on the corresponding objective.

Next, we define some Getter and Setter Methods for our base class, but what exactly are Getters and Setters?

## What are Getter and Setter Methods?
Getter and Setter Methods:

- Getter methods are used to get (retrieve) the value of a private attribute.
- Setter methods are used to set (update) the value of a private attribute.

In programming, we often use getters and setters to protect the access to the data. They allow us to control how a particular attribute is accessed and modified, ensuring that any necessary validation or processing can be performed.

Getter and Setter for `target_value`
- Getter Method: Retrieves the value of target_value.
- Setter Method: Sets a new value for target_value, ensuring it is a number.


```
@property
def target_value(self) -> float:
    return self._target_value

@target_value.setter
def target_value(self, value: float):
    assert isinstance(value, (int, float)), f"target_value must be numerical. Received {type(value)}"
    self._target_value = value

```


Getter and Setter for `weight`
- Getter Method: Retrieves the value of weight.
- Setter Method: Sets a new value for weight, ensuring it is a number.

```
@property
def weight(self) -> float:
    return self._weight

@weight.setter
def weight(self, value: float):
    assert isinstance(value, (int, float)), f"weight must be numerical. Received {type(value)}"
    self._weight = value

```

Summary
- Getter and Setter Methods: These methods allow controlled access to the `target_value` and `weight` attributes, ensuring they are always numerical.
- Usage: The getters retrieve the values, and the setters validate and update the values.

This approach provides a way to manage and safeguard the properties of the ProteinProperty class, ensuring they remain valid throughout their use.

## The Calculate Method
Lastly, we will define an abstract method for our base class. This is the arguably the most important function of the base class, as it will define how we compute the property that we're trying to optimize.

Importantly, the calculate functionality requires a sparrow.Protein object as input - You can read more about SPARROW [here](https://github.com/idptools/sparrow); but one of the most useful methods of the protein object from sparrow is the sequence attribute that enables you to obtain the sequence from the object - e.g., `protein.sequence` would return the sequence of the protein object. For more information, I encourage you to reference the SPARROW codebase and (eventually) documentation.

```
    @abstractmethod
    def calculate(self, protein: 'sparrow.Protein') -> float:
        """
        Calculate the property value for a given protein.

        Parameters:
        protein (sparrow.Protein): The protein instance.

        Returns:
        float: The calculated property value.
        """
        pass

```



# The SequenceOptimizer Class


The `SequenceOptimizer` class provides the interface between the functions that perform the optimization and the property/properties you wish to optimize for this sequence. For more information on these functions, take a look at the documentation (coming soon). For now, we'll focus on demonstrating the utility by example.


```
class SequenceOptimizer:
    def __init__(self, target_length: int, kmer_dict_file: str = None, verbose=False):
        self.target_length = target_length
        self.kmer_dict_file = kmer_dict_file
        self.verbose = verbose
        self.kmer_dict = None
        self.properties: List[ProteinProperty] = []
        self.fixed_ranges: List[Tuple[int, int]] = []
        self.max_iterations = 1000
        self.tolerance = 0.001
        self.window_size = 50
        self.num_shuffles = 5000
        self.shuffle_interval = 25
        self.initial_sequence = None

        # Set up logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)

        # Load kmer_dict if file is provided
        if self.kmer_dict_file:
            self._load_kmer_dict()
        else:
            self.kmer_dict_file = get_data("kmer_properties.pickle")
            self._load_kmer_dict()

    def _load_kmer_dict(self):
        try:
            with open(self.kmer_dict_file, "rb") as f:
                kmer_properties = pickle.load(f)
            self.kmer_dict = build_kmer_dict(kmer_properties)
            self.logger.info(f"Loaded kmer dictionary from {self.kmer_dict_file}")
        except OSError as e:
            self.logger.error(f"Error loading kmer dictionary: {e}")
            raise

    @classmethod
    def load_configuration(cls, config_file: str, kmer_dict_file: str = None):
        with open(config_file, 'r') as f:
            config = json.load(f)

        optimizer = cls(config['target_length'], kmer_dict_file)
        optimizer.set_fixed_ranges(config['fixed_ranges'])
        optimizer.set_optimization_params(
            max_iterations=config['max_iterations'],
            tolerance=config['tolerance'],
            window_size=config['window_size'],
            num_shuffles=config['num_shuffles'],
            shuffle_interval=config['shuffle_interval']
        )
        optimizer.set_initial_sequence(config['initial_sequence'])

        # Load properties
        for prop_name, target_value, weight in config['properties']:
            # properties being optimized must be in global namespace
            property_class = globals()[prop_name]
            optimizer.add_property(property_class, target_value, weight)

        optimizer.logger.info(f"Configuration loaded from {config_file}")
        return optimizer

    def add_property(self, property_class: type, *args: Any, **kwargs: Any):
        self.properties.append(property_class(*args, **kwargs))

    def set_fixed_ranges(self, ranges: List[Tuple[int, int]]):
        self.fixed_ranges = ranges

    def set_optimization_params(self, max_iterations: int = None, tolerance: float = None,
                                window_size: int = None, num_shuffles: int = None,
                                shuffle_interval: int = None):
        if max_iterations is not None:
            self.max_iterations = max_iterations
        if tolerance is not None:
            self.tolerance = tolerance
        if window_size is not None:
            self.window_size = window_size
        if num_shuffles is not None:
            self.num_shuffles = num_shuffles
        if shuffle_interval is not None:
            self.shuffle_interval = shuffle_interval

    def set_initial_sequence(self, sequence: str):
        self.initial_sequence = sequence

    def run(self) -> str:
        self.logger.info("Starting sequence optimization")
        optimized_sequence = optimize_sequence(
            self.kmer_dict,
            self.target_length,
            self.properties,
            max_iterations=self.max_iterations,
            tolerance=self.tolerance,
            verbose=self.verbose,
            window_size=self.window_size,
            num_shuffles=self.num_shuffles,
            shuffle_interval=self.shuffle_interval,
            fixed_residue_ranges=self.fixed_ranges,
            initial_sequence=self.initial_sequence
        )
        self.logger.info("Sequence optimization completed")
        self.log_results(optimized_sequence)
        return optimized_sequence

    def log_results(self, sequence: str):
        if self.initial_sequence is not None:
            self.logger.info("Initial Sequence: " + self.initial_sequence)
        self.logger.info(f"Optimized Sequence: {sequence}")
        protein = sparrow.Protein(sequence)
        for prop in self.properties:
            value = prop.calculate(protein)
            self.logger.info(f"{prop.__class__.__name__}: {value:.2f} (Target: {prop.target_value:.2f})")

    def save_configuration(self, filename: str):
        config = {
            "target_length": self.target_length,
            "properties": [(prop.__class__.__name__, prop.target_value, prop.weight) for prop in self.properties],
            "fixed_ranges": self.fixed_ranges,
            "max_iterations": self.max_iterations,
            "tolerance": self.tolerance,
            "window_size": self.window_size,
            "num_shuffles": self.num_shuffles,
            "shuffle_interval": self.shuffle_interval,
            "initial_sequence": self.initial_sequence
        }
        with open(filename, 'w') as f:
            json.dump(config, f, indent=2)
        self.logger.info(f"Configuration saved to {filename}")
```



# Example 1: Optimizing the fraction of charged residues towards a target value.

First we will instantiate a SequenceOptimizer class. The target length of the optimized sequence is required at this stage.

In [2]:
optimizer = SequenceOptimizer(target_length=100,verbose=False)

INFO:goose.sequence_optimization:Loaded kmer dictionary from /usr/local/lib/python3.10/dist-packages/goose/data/kmer_properties.pickle


Next, we will write a `FCR` subclass of the `ProteinProperty` base class which will provide the interface for us to optimize.

N.B. This class is actually already provided in goose.properties module. You can import it as:

`from goose.properties import FCR`

which we have already done above, so I will simply use that implementation - although it is copied below.

```
class FCR(ProteinProperty):
    """
    Calculate the FCR property.
    """
    def __init__(self, target_value: float, weight: float = 1.0):
        super().__init__(target_value, weight)

    def calculate(self, protein: 'sparrow.Protein') -> float:
        return protein.FCR
```



Next, we will add this property to the list of properties exposed to the `SequenceOptimizer`.

In [3]:
optimizer.add_property(FCR, target_value=0.25, weight=1)

INFO:goose.sequence_optimization:Added new property FCR


To see a list of the properties defined for this optimizer object you can run the following codeblock:

In [4]:
optimizer.get_properties

INFO:goose.sequence_optimization:Defined properties:
INFO:goose.sequence_optimization:  - FCR: target = 0.25, weight = 1


Finally, to run the optimization you can run the entrypoint via `optimizer.run()`

In [5]:
sequence = optimizer.run()

INFO:goose.sequence_optimization:Starting sequence optimization
  0%|          | 1/1000 [00:01<23:58,  1.44s/it]
INFO:goose.sequence_optimization:Sequence optimization completed
INFO:goose.sequence_optimization:Optimized Sequence: QHQQHIIDADADALRGHNHPSDPPILGNVKETETQCRRTSLGVESSPGLPSMDLNTLNHVRGLKQWTSVSTVEKVSSEGFKEEEGSGLYNEKYSMEPRIY
INFO:goose.sequence_optimization:FCR: 0.25 (Target: 0.25)


To save the parameters used for this optimization you can run the following codeblock:

In [6]:
optimizer.save_configuration("test.json")

INFO:goose.sequence_optimization:Configuration saved to test.json


The contents of this json store the parameters used for that optimization object:

In [7]:
cat test.json

{
  "target_length": 100,
  "properties": [
    [
      "FCR",
      0.25,
      1
    ]
  ],
  "fixed_ranges": [],
  "max_iterations": 1000,
  "tolerance": 0.001,
  "window_size": 50,
  "num_shuffles": 5000,
  "shuffle_interval": 25,
  "initial_sequence": null
}

and previously used configurations for optimizers can be reloaded with the following command:

In [9]:
optimizer2 = SequenceOptimizer.load_configuration("test.json")

INFO:goose.sequence_optimization:Loaded kmer dictionary from /usr/local/lib/python3.10/dist-packages/goose/data/kmer_properties.pickle
INFO:goose.sequence_optimization:Added new property FCR
INFO:goose.sequence_optimization:Configuration loaded from test.json


# Example 2: Joint optimization of multiple objectives.

**NOTES:**

1.    Not all possible combinations of objectives are possible, our optimization protocol for arbitrary objectives are greedy - that is, they will try to get as close as possible by minimizing the joint error between the objects [optionally weighted by the weighting factors].
2.   The more computationally expensive and/or orthogonal objectives that are being optimized the more difficult it will be to arrive at a solution within the given tolerance and number of iterations [assuming a solution even exists, see point 1].



In [14]:
optimizer = SequenceOptimizer(target_length=100,verbose=True)
optimizer.set_optimization_params(tolerance=0.1)
optimizer.add_property(FCR,0.2,0.5)
optimizer.add_property(Kappa,0.3,0.3)
optimizer.add_property(RadiusOfGyration,40,0.2)
optimizer.get_properties
optimizer.run()

INFO:goose.sequence_optimization:Loaded kmer dictionary from /usr/local/lib/python3.10/dist-packages/goose/data/kmer_properties.pickle
INFO:goose.sequence_optimization:Added new property FCR
INFO:goose.sequence_optimization:Added new property Kappa
INFO:goose.sequence_optimization:Added new property RadiusOfGyration
INFO:goose.sequence_optimization:Defined properties:
INFO:goose.sequence_optimization:  - FCR: target = 0.2, weight = 0.5
INFO:goose.sequence_optimization:  - Kappa: target = 0.3, weight = 0.3
INFO:goose.sequence_optimization:  - RadiusOfGyration: target = 40, weight = 0.2
INFO:goose.sequence_optimization:Starting sequence optimization
  0%|          | 2/1000 [00:24<2:49:44, 10.20s/it]

Iteration 0: Best Error = 2.2457515846192835
Iteration 0: Best Sequence = FTSQKYNESERGFHGTLSFPTLNQAQSASLSSPANPDEFTSNGQQSIGDRMNKQPGFPPFDHPLLNQSQKSTPGYNHCHDVSDTQRADGGCQRLHCSFRT
Iteration 0: Target FCR= 0.200
Iteration 0: current FCR= 0.170
Iteration 0: Target Kappa= 0.300
Iteration 0: current Kappa= 0.177
Iteration 0: Target RadiusOfGyration= 40.000
Iteration 0: current RadiusOfGyration= 29.031


 10%|█         | 101/1000 [00:43<03:36,  4.15it/s]

Iteration 100: Best Error = 0.9904808098077774
Iteration 100: Best Sequence = DKPAKAHSTTLDSSDTLSKPTLMQATAPIRSSPLNPISPPMSMNPPYLSKVKKSPRKPPIMHPLLSPTCKSTPGSIHPDDSTSKGGPSSTGILRHCEKRT
Iteration 100: Target FCR= 0.200
Iteration 100: current FCR= 0.210
Iteration 100: Target Kappa= 0.300
Iteration 100: current Kappa= 0.234
Iteration 100: Target RadiusOfGyration= 40.000
Iteration 100: current RadiusOfGyration= 34.890


 20%|██        | 202/1000 [01:02<02:03,  6.44it/s]

Iteration 200: Best Error = 0.4848326943814754
Iteration 200: Best Sequence = DKPAKPHNKSLDPHTVLIKPTQPPNNVPIRPKAKKPIRPPMIPNPMIILKVKKSPRKPPIPHPLLRPPLLTQPGSIHPPENAAKGGLLPAVLPQHCEKRT
Iteration 200: Target FCR= 0.200
Iteration 200: current FCR= 0.220
Iteration 200: Target Kappa= 0.300
Iteration 200: current Kappa= 0.252
Iteration 200: Target RadiusOfGyration= 40.000
Iteration 200: current RadiusOfGyration= 36.509


 28%|██▊       | 281/1000 [01:16<03:16,  3.65it/s]
INFO:goose.sequence_optimization:Sequence optimization completed
INFO:goose.sequence_optimization:Optimized Sequence: DKPAKPHPGLVAPIKVLIKPKQPPNNVPIRPKAKKPIRPPMIPLPMPPFPPKKRPRKPPITRPLLRPPMTPTPGSIHPPENAAKMLLLPAVPSPGTEKRT
INFO:goose.sequence_optimization:FCR: 0.23 (Target: 0.20)
INFO:goose.sequence_optimization:Kappa: 0.27 (Target: 0.30)
INFO:goose.sequence_optimization:RadiusOfGyration: 39.65 (Target: 40.00)


'DKPAKPHPGLVAPIKVLIKPKQPPNNVPIRPKAKKPIRPPMIPLPMPPFPPKKRPRKPPITRPLLRPPMTPTPGSIHPPENAAKMLLLPAVPSPGTEKRT'